In [1]:
import tensorflow as tf
import os
import numpy as np
from model import *
from utility import *

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

model_name = "DCGAN_for_MNIST_gpu1"

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
class Generator:
    def __init__(self):
        self.reuse = False
        #self.g_bn0 = BatchNormalization(name = 'g_bn0')

    def __call__(self, z, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            fc0 = full_connection_layer(z, 7*7*128, name="fc0")
            fc0 = tf.nn.relu(fc0)
            fc0 = tf.reshape(fc0, [-1, 7, 7, 128])

            batch_size = tf.shape(fc0)[0]
            deconv0 = deconv2d_layer(fc0, [batch_size, 14, 14, 64], kernel_size=5, name="deconv0")
            deconv0 = tf.nn.relu(deconv0)

            deconv1 = deconv2d_layer(deconv0, [batch_size, 28, 28, 1], kernel_size=5, name="deconv1")
            output = tf.nn.tanh(deconv1)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [3]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.d_bn0 = BatchNormalization(name="d_bn0")
        self.d_bn1 = BatchNormalization(name="d_bn1")
        
    def __call__(self, x,training=False, name=''):
        with tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 28, 28, 1])

            conv1 = conv2d_layer(x, 64, kernel_size=5, name="d_conv0")
            conv1 = lrelu(conv1, leak=0.2)
            
            conv2 = conv2d_layer(conv1, 128, kernel_size=5, name="d_conv1")
            conv2 = self.d_bn0(conv2)
            conv2 = lrelu(conv2, leak=0.2)
            conv2 = tf.contrib.layers.flatten(conv2)

            fc0 = full_connection_layer(conv2, 1, name="fc0")
            disc = fc0

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')

        return disc

In [4]:
class GAN:
    def __init__(self):
        self.batch_size = 128
        self.img_size = 28
        self.z_size = 50
        
        self.epochs = 50000
        self.epoch_saveMetrics = 1000
        self.epoch_saveSampleImg = 1000
        self.epoch_saveParamter = 5000
        self.losses = {"d_loss":[], "g_loss":[]}

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 1])
        self.z = tf.placeholder(tf.float32, [None, self.z_size])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.z)

    def loss(self):
        disc_tr = self.d(self.X_tr)
        disc_gen = self.d(self.Xg)
        
        loss_d_tr = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_tr, labels=tf.ones_like(disc_tr)))
        loss_d_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen, labels=tf.zeros_like(disc_gen)))
        loss_d = (loss_d_tr + loss_d_gen)/2
        
        loss_g = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=disc_gen, labels=tf.ones_like(disc_gen)))
        
        return loss_g, loss_d

    def train(self):
        # Optimizer
        d_lr = 2e-4
        d_beta1 = 0.5
        g_lr = 2e-4
        g_beta1 = 0.5

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.AdamOptimizer(learning_rate=d_lr)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=g_lr)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list= "1"
            )
        )
                
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())

            # preparing noise vec for test
            bs = 100
            test_z = np.random.uniform(-1, 1, size=[bs, self.z_size])

            for epoch in range(self.epochs):              
                # extract images for training
                X_mb, _ = mnist.train.next_batch(self.batch_size)
                X_mb = np.reshape(X_mb, [-1, 28, 28, 1])

                z = np.random.uniform(-1, 1, size=[self.batch_size, self.z_size])

                # train Discriminator
                _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # train Generator
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # append loss value for visualizing
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                # print epoch
                if epoch % 50 == 0:
                    print('epoch:{0}, d_loss:{1}, g_loss{2} '.format(epoch, d_loss_value, g_loss_value))
                
                # visualize loss
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(model_name, self.losses, epoch)

                # visualize generated images during training
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.z: test_z})
                    save_imgs(model_name, img, name=str(epoch))

                # save model parameters 
                if epoch % self.epoch_saveParamter == 0:
                    dir_path = "model_" + model_name
                    if not os.path.isdir(dir_path):
                        os.makedirs(dir_path)

                    saver.save(sess, dir_path + "/" + str(epoch) + ".ckpt")

In [5]:
gan = GAN()
gan.train()

/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:50: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


epoch:0, d_loss:0.670637845993042, g_loss0.689831018447876 
epoch:50, d_loss:0.0013159073423594236, g_loss8.670087814331055 
epoch:100, d_loss:0.0013340885052457452, g_loss7.395753383636475 
epoch:150, d_loss:0.0023037861101329327, g_loss8.287544250488281 
epoch:200, d_loss:0.00258297985419631, g_loss7.169459819793701 
epoch:250, d_loss:0.0006026382907293737, g_loss8.486684799194336 
epoch:300, d_loss:0.012050358578562737, g_loss7.045624732971191 
epoch:350, d_loss:0.001091190380975604, g_loss8.24703311920166 
epoch:400, d_loss:0.006045861169695854, g_loss9.056556701660156 
epoch:450, d_loss:0.0015668777050450444, g_loss8.36926555633545 
epoch:500, d_loss:0.0021768901497125626, g_loss8.738700866699219 
epoch:550, d_loss:0.00030371849425137043, g_loss8.619287490844727 
epoch:600, d_loss:0.0003375725937075913, g_loss9.421628952026367 
epoch:650, d_loss:0.0042360275983810425, g_loss8.353374481201172 
epoch:700, d_loss:0.0011783769587054849, g_loss7.812603950500488 
epoch:750, d_loss:0.003

KeyboardInterrupt: 